<a href="https://colab.research.google.com/github/A-THAKUR22/MarchAssignments/blob/main/Assignment3March8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=0ec608d655e56ada6452f268c478732c43d71a563e8a77b1e6341b7e137ee04a
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


Load the Walmart Stock CSV File, have Spark infer the data types.

In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Walmart Analysis").getOrCreate()
stock_df=spark.read.csv("walmart_stock.csv", header=True, inferSchema=True)
stock_df

DataFrame[Date: date, Open: double, High: double, Low: double, Close: double, Volume: int, Adj Close: double]

What are the column names?

In [6]:
stock_df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

What does the Schema look like?

In [8]:
stock_df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



Print out the first 5 columns.

In [11]:
first_5=stock_df.select(stock_df.columns[:5]).show(5)
print(first_5)

+----------+------------------+---------+---------+------------------+
|      Date|              Open|     High|      Low|             Close|
+----------+------------------+---------+---------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18|
+----------+------------------+---------+---------+------------------+
only showing top 5 rows

None


Use describe() to learn about the DataFrame.

In [15]:
stock_df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar.


In [22]:
from pyspark.sql.functions import format_number
descr=stock_df.describe()
formatted=descr.select(descr['summary'],format_number(descr['Open'].cast('float'),2).alias('Open'),
                       format_number(descr['High'].cast('float'),2).alias('High'),
                       format_number(descr['Low'].cast('float'),2).alias('Low'),
                       format_number(descr['Close'].cast('float'),2).alias('Close'),
                       format_number(descr['Volume'].cast('float'),2).alias('Volume')).show()


+-------+--------+--------+--------+--------+-------------+
|summary|    Open|    High|     Low|   Close|       Volume|
+-------+--------+--------+--------+--------+-------------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,096.00|
+-------+--------+--------+--------+--------+-------------+



Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day

In [26]:
from pyspark.sql.functions import col
hv_ratio=stock_df.withColumn('HV Ratio',col('High')/col('Volume'))
hv_ratio.select("HV Ratio").show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



What day had the Peak High in Price?

In [31]:
from pyspark.sql.functions import desc
Highest_price_day=stock_df.orderBy(desc("High")).select("Date").first()[0]
Highest_price_day

datetime.date(2015, 1, 13)

What is the max and min of the Volume column?

In [41]:
from pyspark.sql.functions import max,min
max_min_vol=stock_df.select(max("Volume"),min("Volume"))
max_min_vol.show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



How many days was the Close lower than 60 dollars?

In [35]:
from pyspark.sql.functions import col
days_lower_close_60=stock_df.filter(col("Close")<60).count()
print(days_lower_close_60)

81


What percentage of the time was the High greater than 80 dollars?

In [38]:
from pyspark.sql.functions import col
Higher_price_80=stock_df.filter(col("High")>80).count()
total=stock_df.count()
percentage=(Higher_price_80/total)*100
print(round(percentage,6))

9.141494


What is the max High per year

In [39]:
from pyspark.sql.functions import max,year

df_Years=stock_df.withColumn('Year',year(stock_df['Date']))
highest_of_all_years=df_Years.groupBy('Year').agg(max('High').alias('MaxHighAllYears'))
highest_of_all_years.show()

+----+---------------+
|Year|MaxHighAllYears|
+----+---------------+
|2015|      90.970001|
|2013|      81.370003|
|2014|      88.089996|
|2012|      77.599998|
|2016|      75.190002|
+----+---------------+



What is the Pearson correlation between High and Volume?

In [44]:
from pyspark.sql.functions import corr
correlation_high_vol=stock_df.corr('High','Volume')
correlation_high_vol

-0.3384326061737161

What is the average Close for each Calendar Month?
In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months.


In [46]:
from pyspark.sql.functions import month,avg
df_month=stock_df.withColumn('Months',month(stock_df['Date']))
new=df_month.groupby('Months').agg(avg('Close').alias('Average Closing Price'))
monthly_closing_price=new.orderBy('Months')
monthly_closing_price.show()

+------+---------------------+
|Months|Average Closing Price|
+------+---------------------+
|     1|    71.44801958415842|
|     2|      71.306804443299|
|     3|    71.77794377570092|
|     4|    72.97361900952382|
|     5|    72.30971688679247|
|     6|     72.4953774245283|
|     7|    74.43971943925233|
|     8|    73.02981855454546|
|     9|    72.18411785294116|
|    10|    71.57854545454543|
|    11|     72.1110893069307|
|    12|    72.84792478301885|
+------+---------------------+

